In [1]:
import sys
sys.path.append(r"D:\Scripts1\Code\ActPEA\CODE\Actu_Colums.py")
import pandas as pd
import matplotlib as mptl
import Actu_Colums as Ac
from openpyxl import load_workbook
from typing import Optional
import glob,os,re
import numpy as np
from collections import defaultdict
from babel.dates import format_date, format_datetime, Locale
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
import pickle

## Funciones

In [2]:
def mes_a_numero(mes):
    meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
             'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    return meses.index(mes) + 1 if mes in meses else None
def parse_fecha(fecha):
    for fmt in ('%Y-%m-%d', '%d-%m-%Y'):
        try:
            return datetime.strptime(fecha, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')
def normalize_company_names(df, column):
    """Normaliza los nombres de las empresas en la columna especificada del DataFrame."""
    # Reemplaza "SAC" o "S. A. C." al final de los nombres de las empresas con "S.A.C"
    df[column] = df[column].str.replace(r"(SAC|S\. ?A\. ?C\.)$", "S.A.C.", regex=True)
    return df
def relacionar_dataframes(df_hechos, df_dimensiones, columna_hechos, columna_dimensiones):
    # Renombrar la columna en df_dimensiones para que coincida con df_hechos
    df_dimensiones_renombrado = df_dimensiones.rename(columns={columna_dimensiones: columna_hechos})
    
    # Unir los dataframes
    df_resultado = pd.merge(df_hechos, df_dimensiones_renombrado, on=columna_hechos,how='left')
    df_resultado.drop(columns=[columna_hechos],inplace=True)
    rows_in = len(df_hechos)
    rows_out = len(df_resultado)
    diff_row = rows_in-rows_out
    print(f"el df entra con: {rows_in}. y sale con {rows_out},hay una diferencia de {diff_row} filas")
    
    return df_resultado
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 
         7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}


# El esquema que voy a seguir es:
-  https://miro.com/welcomeonboard/R091Q0xPdE13YUNNbzBWTEl4b01CYnBjQmNuWG9sVW9qa2NLN1dIWTludmlIZUFIcmc5c0hLTWhGdk5HSW5mb3wzNDU4NzY0NTkxMzU2NDUxNTY0fDI=?share_link_id=576727954374

## Cargo las 4 tablas 

In [3]:
prepa_f = pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Reporte Prepasivo.csv')
prepa_ts = pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Prepa_TS.csv')
EA_f = pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\EAS\EAs.csv')
EA_ts = pd.read_csv(r'D:\EA\TS\EA_TS1.csv') 

### Preproces

In [4]:
##### TIME Preprocess #############3
prepa_ts = prepa_ts.dropna(subset='PEND_FACT_SOLES')
prepa_ts.TIME =  prepa_ts.TIME.str.replace('/','-')
mask = prepa_ts.TIME.str.split('-').str[0].str.len() == 4
# Usamos la máscara para aplicar el formato correcto a cada subconjunto los que tiene  primero el año o el dia.
prepa_ts.loc[mask, 'TIME'] = pd.to_datetime(prepa_ts.loc[mask, 'TIME'], format='%Y-%m-%d').dt.date #
prepa_ts.loc[~mask, 'TIME'] = pd.to_datetime(prepa_ts.loc[~mask, 'TIME'], format='%d-%m-%Y').dt.date

prepa_ts['TIME'] = pd.to_datetime(prepa_ts['TIME'])
EA_ts['TIME'] = pd.to_datetime(EA_ts['TIME'])
EA_ts['Mes'] = pd.to_datetime(EA_ts['Fecha_DOC']).dt.month.map(meses) ## Convierto a mes las fechas_DOC
EA_f['Mes'] = pd.to_datetime(EA_f['FECHA_DOC']).dt.month.map(meses) ## Convierto a mes las fechas_DOC

In [5]:
############# Responsable Preprocess #####################
EA_ts['RESPONSABLE EA'] = EA_ts['RESPONSABLE EA'].str.upper().str.split(' ',expand=True)[0]
EA_ts['RESPONSABLE EA'].replace('ANNGIE','ANGGIE',inplace=True)
EA_ts['RESPONSABLE EA'].replace('POR','POR ASIGNAR',inplace=True)
prepa_f['Responsable3'] = prepa_f['Responsable3'].str.upper()


In [6]:
############## Contrata Preprocesss #####################
EA_f = normalize_company_names(EA_f,'PROVEEDOR')

EA_f['PROVEEDOR'] = EA_f['PROVEEDOR'].str.upper()
prepa_ts = normalize_company_names(prepa_ts,'NOMPROVEEDOR')
prepa_ts.replace({'COMFUTURA':'COMUNICACION FUTURA SOCIEDAD ANONIM',
                       'SCALA DISEÃ‘O Y CONSTRUCCION S.A.C.':'SCALA DISEÑO Y CONSTRUCCION S.A.C.'}
                       ,inplace=True)

prepa_f.Contratista = prepa_f.Contratista.str.upper()
prepa_f = normalize_company_names(prepa_f,'Contratista')
prepa_f.replace({'COMFUTURA':'COMUNICACION FUTURA SOCIEDAD ANONIM',
                       'SCALA DISEÃ‘O Y CONSTRUCCION S.A.C.':'SCALA DISEÑO Y CONSTRUCCION S.A.C.'}
                       ,inplace=True)

In [7]:
##################  Estado Preprocess ####################### 
prepa_ts['ESTADO'] = prepa_ts['ESTADO'].str.upper()
prepa_f['Estado'] = prepa_f['Estado'].str.upper()

### Dimension MES (mes_dim)
### Se relaciona con:
- EA_f : Mes
- EA_ts : Mes
- prepa_ts : MES-COMPROMISO
- prepa_f : Mes de Compromiso

In [8]:
# Concatenate the months from the fact tables and remove duplicates
mes_dim = pd.concat([prepa_f['Mes De Compromiso'], EA_f['Mes'], EA_ts['Mes'], prepa_ts['MES-COMPROMISO']]).drop_duplicates().reset_index(drop=True)

# Create a DataFrame with an index and replace 'Setiembre' with 'Septiembre'
mes_dim = pd.DataFrame(mes_dim, columns=['Mes']).reset_index().rename(columns={'index': 'Mes_id'}).replace('Setiembre', 'Septiembre')

# Remove duplicates based on 'Mes' and apply the 'mes_a_numero' function
mes_dim = mes_dim.drop_duplicates(subset='Mes').assign(OrdenMes=lambda x: x['Mes'].apply(mes_a_numero)).dropna(subset=['Mes'])

# Replace 'Setiembre' with 'Septiembre' in the specified columns
prepa_f.replace('Setiembre', 'Septiembre', inplace=True)
prepa_ts['MES-COMPROMISO'].replace('Setiembre', 'Septiembre', inplace=True)



In [9]:
mes_dim

,Mes_id,Mes,OrdenMes
0,0,Julio,7.0
1,1,Agosto,8.0
2,2,Septiembre,9.0
3,3,Bloqueo de AF,NaN
4,4,Enero,1.0
5,5,Febrero,2.0
6,6,Marzo,3.0
7,7,Abril,4.0
8,8,Mayo,5.0
9,9,Junio,6.0


In [10]:
# Relate dataframes
prepa_f_r = relacionar_dataframes(prepa_f, mes_dim[['Mes', 'Mes_id']], 'Mes De Compromiso', 'Mes')
EA_f_r = relacionar_dataframes(EA_f, mes_dim[['Mes', 'Mes_id']], 'Mes', 'Mes')
prepa_ts_r = relacionar_dataframes(prepa_ts, mes_dim[['Mes', 'Mes_id']], 'MES-COMPROMISO', 'Mes')
EA_ts_r = relacionar_dataframes(EA_ts, mes_dim[['Mes', 'Mes_id']], 'Mes', 'Mes')

# Drop unnecessary columns
EA_f_r.drop(columns=['FECHA_DOC'], inplace=True)
prepa_ts_r.drop(columns=['TIME Format'], inplace=True)
EA_ts_r.drop(columns=['Fecha_DOC', 'DATE_FORMAT'], inplace=True)

el df entra con: 179. y sale con 179,hay una diferencia de 0 filas
el df entra con: 1297. y sale con 1297,hay una diferencia de 0 filas
el df entra con: 8136. y sale con 8136,hay una diferencia de 0 filas
el df entra con: 1987. y sale con 1987,hay una diferencia de 0 filas


### Dimension Analista (Analista_dim)
### Se relaciona con: (REDY)
    - EA_ts : RESPONSABLE EA
    - EA_F : RESPONSABLE_DE_EA
    -prepa_f : Responsable3

In [11]:
Analista_dim = pd.concat([prepa_f['Responsable3'],EA_f['RESPONSABLE_DE_EA'],EA_ts['RESPONSABLE EA']])\
    .drop_duplicates()\
    .reset_index(drop=True)\
    .reset_index()\
    .rename(columns={'index': 'analista_id', 0: 'Analista'})\
    .replace({'Analista': {'ANNGIE': 'ANGGIE'}})\
    .drop_duplicates(subset='Analista')
  
prepa_f_r = relacionar_dataframes(prepa_f_r,Analista_dim,'Responsable3','Analista')#prepa_F
EA_f_r = relacionar_dataframes(EA_f_r,Analista_dim,'RESPONSABLE_DE_EA','Analista')#EA_f_r
EA_ts_r = relacionar_dataframes(EA_ts_r,Analista_dim,'RESPONSABLE EA','Analista') # EA_ts 

el df entra con: 179. y sale con 179,hay una diferencia de 0 filas
el df entra con: 1297. y sale con 1297,hay una diferencia de 0 filas
el df entra con: 1987. y sale con 1987,hay una diferencia de 0 filas


### Dimension Contratista (Contra_dim_NP)
### Se relaciona con: 
    -EA_f : PROVEEDOR
    -Prepa_ts : NOMPROVEEDOR
    -Prepa_f : Contratista

In [12]:
Contra_dim_NP = pd.concat([prepa_f['Contratista'],EA_f['PROVEEDOR'],prepa_ts['NOMPROVEEDOR']])\
        .str.upper()\
        .reset_index(drop=True)\
        .reset_index()\
        .rename(columns={'index': 'contrata_id', 0: 'Contrata'})

Contra_dim_NP = normalize_company_names(Contra_dim_NP,'Contrata')
Contra_dim_NP.drop_duplicates(subset='Contrata',inplace=True)
Contra_dim_NP.dropna(subset='Contrata',inplace=True)

EA_f_r = relacionar_dataframes(EA_f_r,Contra_dim_NP,'PROVEEDOR','Contrata') 
prepa_ts_r = relacionar_dataframes(prepa_ts_r,Contra_dim_NP,'NOMPROVEEDOR','Contrata')
prepa_f_r = relacionar_dataframes(prepa_f_r,Contra_dim_NP,'Contratista','Contrata')

el df entra con: 1297. y sale con 1297,hay una diferencia de 0 filas
el df entra con: 8136. y sale con 8136,hay una diferencia de 0 filas
el df entra con: 179. y sale con 179,hay una diferencia de 0 filas


### Dimension ESTADO Prepa(Estado_Prepa_dim_S)
### Se relaciona con:
    -Prepa_ts: Estado
    -Prepa_f: Estado

In [13]:
Estado_sort = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\Estados_N.txt')  # Cargo el orden de los estados  
Estado_sort.rename(columns={'Estado_Sort':'Estado'},inplace=True)

Estado_Prepa_dim = pd.concat([prepa_f['Estado'],prepa_ts['ESTADO']])\
                .drop_duplicates().reset_index(drop=True).reset_index(drop=False)\
                .rename(columns={'index': 'Estado_id', 0: 'Estado'})

Estado_Prepa_dim_S = pd.merge(Estado_Prepa_dim,Estado_sort,on='Estado',how='left') # añado orden a mis estados actuales 

In [14]:
prepa_ts_r = relacionar_dataframes(prepa_ts_r,Estado_Prepa_dim_S[['Estado_id','Estado']],'ESTADO','Estado')
prepa_f_r = relacionar_dataframes(prepa_f_r,Estado_Prepa_dim_S[['Estado_id','Estado']],'Estado','Estado')

el df entra con: 8136. y sale con 8136,hay una diferencia de 0 filas
el df entra con: 179. y sale con 179,hay una diferencia de 0 filas


### Dimension Estado de EA(Estado_EA_dim)
###  Se relaciona con:
    - EA_ts: ESTADO EA
    - EA_f: Estado EA

In [15]:
Estado_EA_dim = pd.concat([EA_ts['ESTADO EA'],EA_f['Estado de EA']]).drop_duplicates().reset_index(drop=True).reset_index(drop=False)\
                .rename(columns={'index': 'Estado_EA_id', 0: 'Estado_EA'})

In [16]:
EA_ts_r = relacionar_dataframes(EA_ts_r,Estado_EA_dim,'ESTADO EA','Estado_EA')

el df entra con: 1987. y sale con 1987,hay una diferencia de 0 filas


In [17]:
EA_f_r = relacionar_dataframes(EA_f_r,Estado_EA_dim,'Estado de EA','Estado_EA')

el df entra con: 1297. y sale con 1297,hay una diferencia de 0 filas


### Tabla Calendario(Calendar_dim)
#### Se relaciona con: 
    - Prepa_ts : TIME
    - EA_ts : TIME

In [18]:
date_concat = pd.concat([prepa_ts['TIME'],EA_ts['TIME']]).drop_duplicates().reset_index(drop=True).reset_index(drop=False)
##:Concateno, para crear cotas temporales


In [19]:
date_concat['TIME'] = pd.to_datetime(date_concat['TIME'])
max_date= date_concat['TIME'].max()
min_date= date_concat['TIME'].min()
# Crea un rango de fechas
date_range = pd.date_range(start=min_date, end=max_date)

# Crea un DataFrame a partir del rango de fechas
Calendar_dim = pd.DataFrame(date_range, columns=['Date'])
Calendar_dim = Calendar_dim.reset_index().rename(columns={'index': 'Date_id'})# Este es perma

In [20]:
EA_ts_r = relacionar_dataframes(EA_ts_r,Calendar_dim,'TIME','Date')

el df entra con: 1987. y sale con 1987,hay una diferencia de 0 filas


In [21]:
prepa_ts_r = relacionar_dataframes(prepa_ts_r,Calendar_dim,'TIME','Date')

el df entra con: 8136. y sale con 8136,hay una diferencia de 0 filas


### Tablas de Hechos : 

In [22]:
prepa_f_r.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Prepa_data\Prepa_f.csv',index=False)
prepa_ts_r.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Prepa_data\Prepa_ts.csv',index=False)
EA_f_r.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\EAs_data\EA_f.csv',index=False)
EA_ts_r.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\EAs_data\EA_ts.csv',index=False)


### Dimensiones : Calendar_dim, Estado_EA_dim, Estado_Prepa_dim_S, Contra_dim_NP, Analista_dim, mes_dim

In [23]:
Calendar_dim.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\Calendar_dim.csv',index=False)
Estado_EA_dim.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\Estado_EA_dim.csv',index=False)
Estado_Prepa_dim_S.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\Estado_Prepa_dim_S.csv',index=False)
Contra_dim_NP.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\Contra_dim_NP.csv',index=False)
Analista_dim.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\Analista_dim.csv',index=False)
mes_dim.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\mes_dim.csv',index=False)
